In [1]:
from query import query_high_low_quantile, calculate_standardized_margin_comparison_quallent
import polars as pl
from polars import col as c
import polars.selectors as cs
import warnings
warnings.filterwarnings('ignore')

## Purpose

Identify trends in pharmacy margin using NADAC as an estimated cost basis.

## Key questions
- Are there systematic differences in reimbursement between PBM-affiliated and non‑affiliated pharmacies?
- Which providers lie in the tails of the margin distribution (1st and 99th percentiles)?

## Data sources
- WV OIG PBM NADAC Reporting
    - Contains pharmacy claims data submitted by PBMs to the West Virginia Offices of the Insurance Commissioner (OIC) and Office of the Inspector General (OIG). Each record includes claim-level details such as total reimbursement paid to the pharmacy, the CMS-published National Average Drug Acquisition Cost (NADAC) for the dispensed drug, pharmacy affiliation status, National Drug Code (NDC), drug name, and date of service. The dataset is used to assess pharmacy reimbursement relative to NADAC and to identify potential outliers or trends in pharmacy margins.
- CMS NADAC Reporting  
    - The National Average Drug Acquisition Cost (NADAC) is a federal price benchmark published weekly by CMS for prescription drugs reimbursed by Medicaid. NADAC reflects the national average invoice price paid by retail community pharmacies to acquire prescription and over-the-counter drugs. It is intended to represent the typical acquisition cost for pharmacies, excluding rebates, discounts, or special pricing arrangements. More information is available at [medicaid.gov/nadac](https://www.medicaid.gov/medicaid/nadac).



## Definitions
- **Pharmacy margin** = `total - nadac` (positive => paid more than NADAC; negative => paid below NADAC).
- **affiliate** = whether the pharmacy is affiliated with the PBM (as provided in the dataset).

## Methods (summary)
1. Compute `margin = total - nadac` at the claim level.
2. Group results by `affiliate` and compare tails (1st/99th percentiles) to see how the distribution differs.
3. Limit to the most recent last 2 full years (2023 and 2024)

## Outputs
- Summary table by `affiliate`: count, median margin, 1st/99th percentiles.
- Visualizations: distribution plots and time series of margins.
- CSV with top N outlier providers for follow-up.

## Caveats
- NADAC is an estimate of acquisition cost and may not reflect discounts, rebates, or special pricing agreements.

### Analysis Comparing the Least and Most Profitable Prescriptions - Affiliated vs Non-Affiliated Pharmacies

In [2]:
query_high_low_quantile(0.01,0.99).collect(engine='streaming').to_pandas()

,affiliate,low,high,net,%_diff
0,False,-27.69,146.69,119.00,NaN
1,True,-119.50,445.43,325.93,1.74


### Query High and Low Quantiles

- **Affiliated pharmacies** experienced substantially larger losses on the least profitable prescriptions (1st percentile margin: **-119.50**) compared to non-affiliated pharmacies (**-27.69**).
- On the most profitable prescriptions (99th percentile), affiliated pharmacies achieved much higher average margins (**445.43**) than non-affiliated pharmacies (**146.69**).
- The net difference between the 99th and 1st percentile margins was significantly greater for affiliated pharmacies (**325.93** vs **119.00**), representing a **174% net improvement** over non-affiliated pharmacies.

These findings suggest that affiliated pharmacies are exposed to both greater downside and upside in reimbursement, with a markedly higher net gain at the extremes.

# Are there cases where PBM private label products have higher margins than non-private label?

### Background
- Evernorth is a health services division of Cigna, and Quallent is a pharmacy services subsidiary under Evernorth. Quallent operates as a private label pharmacy benefit manager (PBM) platform, providing PBM services and solutions often integrated with Evernorth’s broader health services.

### Objective
- Identify instances where PBM private-label products (Quallent-managed NDCs/GPIs) show higher standardized margins than non-private-label products for the same GPI.

#### Contract / Quick contract (inputs, outputs, success):
- Inputs: claims dataset (loaded by `load_data()`), NADAC/unit_price, `gpi`, NDC-level flags (Quallent), and quantity columns used by `standardize_margin()`.
- Output: a table with one row per `gpi` containing median standardized margin for Quallent NDCs and non-Quallent NDCs, the difference (qualient - other), median quantity, median_nadac, and generic_name.
- Success: rows where `difference` is positive indicate GPIs where Quallent private-label products have higher median standardized margins than their non-private-label counterparts.

#### Methods (detailed)
1. Filter to ESI reportings and GPI scope related to Quallent (the query helper will apply `is_esi_reporting()` and `is_quallent_gpis()`).
2. Flag claims that are Quallent NDCs via `is_quallent_ndcs()`.
3. Compute a standardized margin per GPI with `standardize_margin()` so differences are comparable across pack sizes/qtys.
4. Aggregate to median standardized margin at the GPI level separately for Quallent and non‑Quallent NDCs, then pivot so both groups appear on the same row and compute `difference = qualient - other`.
5. Join product metadata (generic name) and median NADAC to aid interpretation, then sort by `difference` descending to highlight GPIs where Quallent products have the largest positive margin advantage.

#### How to interpret results
- `difference > 0`: Quallent private-label products have higher median standardized margins for that GPI.
- `median_nadac` and `median_qty` provide scale context; prioritize GPIs with meaningful `total_gpi`/volume for follow-up.

#### Edge cases and caveats
- Only GPIs with observations for both Quallent and non-Quallent NDCs are returned (the query filters out others).
- Standardization aims to normalize unit differences, but check raw totals and volume to avoid over-emphasizing low-volume outliers.
- NADAC may not reflect rebates or special pricing; use results as signals for investigation, not definitive proof of overpayment.



In [3]:
# Example: preview top GPIs where Quallent (private-label) has higher standardized margins
# The function returns a lazy frame; we head() then collect to limit memory and stream the results.
df_qualient = calculate_standardized_margin_comparison_quallent().head(10).collect(engine='streaming')
# show the top results in a pandas-friendly view
df_qualient.to_pandas()

,gpi,median_qty,true,false,difference,generic_name,median_nadac
0,21-53-18-35-10-03-20,105.0,5257.64,1422.67,3834.97,Imatinib Mesylate Tab 100 MG (Base Equivalent),63.92
1,40-14-30-60-10-03-20,45.0,233.41,97.93,135.48,Sildenafil Citrate Tab 20 MG,3.15
2,40-30-40-80-00-03-20,6.0,94.85,10.71,84.14,Tadalafil Tab 20 MG,1.58
3,58-18-00-25-10-67-40,90.0,265.30,192.06,73.24,Duloxetine HCl Enteric Coated Pellets Cap 40 M...,120.13
4,40-30-40-80-00-03-05,30.0,82.10,11.48,70.62,Tadalafil Tab 5 MG,4.41
5,72-60-00-43-00-03-50,180.0,52.13,39.49,12.64,Levetiracetam Tab 1000 MG,34.87
6,21-53-18-35-10-03-40,30.0,5412.67,5401.31,11.36,Imatinib Mesylate Tab 400 MG (Base Equivalent),56.37
7,49-27-00-40-00-65-20,90.0,41.99,34.44,7.55,Lansoprazole Cap Delayed Release 30 MG,10.00
8,72-60-00-30-00-03-40,90.0,13.22,6.69,6.53,Gabapentin Tab 800 MG,10.54
9,27-25-00-50-00-75-30,90.0,17.04,10.57,6.47,Metformin HCl Tab ER 24HR 750 MG,5.90
